# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [10]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [6]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [7]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [14]:
for col in salaries.columns:
    print(f"Column: {col}")
    print(salaries[col].value_counts())
    print("-" * 30)

Column: Name
Name
HERNANDEZ,  JUAN C      4
KELLY,  MICHAEL J       4
DELGADO,  JUAN          4
RIVERA,  RICARDO        4
ROMERO,  MIGUEL A       4
                       ..
GREEN,  CHANDREA N      1
GREEN,  CAROLYN         1
GREEN,  CARLA T         1
GREEN,  CARLA           1
ZYSKOWSKI,  DARIUSZ     1
Name: count, Length: 32880, dtype: int64
------------------------------
Column: Job Titles
Job Titles
POLICE OFFICER                            9520
FIREFIGHTER-EMT                           1473
SERGEANT                                  1202
POOL MOTOR TRUCK DRIVER                   1055
POLICE OFFICER (ASSIGNED AS DETECTIVE)     989
                                          ... 
INQUIRY AIDE I                               1
SECRETARY TO THE CHAIRMAN                    1
CHIEF CONSTRUCTION EQUIPMENT INSPECTOR       1
CLAIMS MANAGER                               1
MECHANICAL ENGINEER IV                       1
Name: count, Length: 1111, dtype: int64
------------------------------
Column

# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [11]:
hourly_wages = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate'].dropna()

# Perform one-sample t-test
t_statistic, p_value = stats.ttest_1samp(hourly_wages, 30)

# Define significance level
alpha = 0.05

# Print the results
print(f"T-Statistic: {t_statistic:.4f}")
print(f"P-Value: {p_value:.4f}")

# Compare p-value with alpha
if p_value < alpha:
    print("Reject null hypothesis: The mean hourly wage is significantly different from $30/hour.")
else:
    print("Fail to reject null hypothesis: The mean hourly wage is not significantly different from $30/hour.")

T-Statistic: 20.6198
P-Value: 0.0000
Reject null hypothesis: The mean hourly wage is significantly different from $30/hour.


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [12]:
# Calculate sample statistics
mean_hourly_wage = hourly_wages.mean()
std_error_hourly_wage = stats.sem(hourly_wages)  # Calculate standard error of the mean

# Calculate degrees of freedom
df = len(hourly_wages) - 1

# Calculate 95% confidence interval using t-distribution
confidence_level = 0.95
confidence_interval = stats.t.interval(confidence_level, df, loc=mean_hourly_wage, scale=std_error_hourly_wage)

# Print results
print(f"Mean Hourly Wage: ${mean_hourly_wage:.2f}")
print(f"Standard Error of the Mean: {std_error_hourly_wage:.4f}")
print(f"95% Confidence Interval: (${confidence_interval[0]:.2f}, ${confidence_interval[1]:.2f})")


Mean Hourly Wage: $32.79
Standard Error of the Mean: 0.1352
95% Confidence Interval: ($32.52, $33.05)


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [16]:
from statsmodels.stats.proportion import proportions_ztest
# Calculate the number of hourly workers in Chicago
hourly_workers_chicago = salaries[(salaries['Salary or Hourly'] == 'Hourly') & (salaries['Department'] == 'POLICE')].shape[0]

# Calculate the total number of workers in Chicago
total_workers_chicago = salaries[salaries['Department'] == 'POLICE'].shape[0]

# Calculate the proportion of hourly workers in Chicago
p_hat = hourly_workers_chicago / total_workers_chicago

# Define the null hypothesis value (expected proportion)
null_hypothesis_value = 0.25  # 25%

# Perform z-test for proportions
z_score, p_value = proportions_ztest(hourly_workers_chicago, total_workers_chicago, value=null_hypothesis_value)

# Define significance level
alpha = 0.05

# Print results
print(f"Proportion of hourly workers in Chicago: {p_hat:.4f}")
print(f"Z-score: {z_score:.4f}")
print(f"P-value: {p_value:.4f}")

# Compare p-value with alpha
if p_value < alpha:
    print("Reject null hypothesis: The proportion of hourly workers in Police Department is significantly different from 25%.")
else:
    print("Fail to reject null hypothesis: The proportion of hourly workers in Police Department is not significantly different from 25%.")

Proportion of hourly workers in Chicago: 0.0007
Z-score: -1057.7019
P-value: 0.0000
Reject null hypothesis: The proportion of hourly workers in Chicago is significantly different from 25%.
